In [ ]:
# Import standard libraries
import pandas as pd
import numpy as np
import math
import bs4 as bs
import urllib.request
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

# Import for text analytics
import spacy
from spacy import displacy
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
import wikipedia

# Text Analytics

<img src='https://images.unsplash.com/photo-1605429201125-37e867327609?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8&auto=format&fit=crop&w=1176&q=80' width="450">

Credit: [Piotr Łaskawski](https://unsplash.com/@tot87)

## Content

The goal of this walkthrough is to provide you with insights on text analytics. [Text Analytics](https://en.wikipedia.org/wiki/Text_mining) (or text mining) is "the discovery by computer of new, previously unknown information, by automatically extracting information from different written resources." ([Marti Hearst](https://people.ischool.berkeley.edu/~hearst/text-mining.html)). Written resources may include websites, books, emails, reviews, and articles.

There are many applications of text analytics, for example:
- Search for relevant websites or articles using a search engine;
- Sentiment Analysis (e.g., classify tweets or film reviews as positive, neutral or negative);
- Summarize, anonymize, or translate documents;
- Chatbots (e.g., ChatGPT, Siri, Alexa);
- etc.

In this notebook, we will see how to prepare and represent texts and explore various text-analytics techniques, before doing an application on text similarity:
- [Text Preparation](#Text-Preparation)
    - [Tokenization](#Tokenization)
    - [Remove Stopwords](#Remove-Stopwords)
    - [Lemmatization](#Lemmatization)
    - [Your turn!](#Your-turn-Preparation)
- [Text structure](#Text-structure)
    - [Dependency Parsing](#Dependency-Parsing)
    - [Entity Detection](#Entity-Detection)
    - [Your turn!](#Your-turn-structure)
- [Text Representation](#Text-Representation)
    - [Bag of Words (BOW)](#Bag-of-Words-(BOW))
    - [TF-IDF Representation](#TF-IDF-Representation)
    - [Your turn!](#Your-turn-Representation)
- [Application: Text Similarity](#Application:-Text-Similarity)
    - [Converting text into vectors](#Converting-text-into-vectors)
    - [Cosine similarity](#Cosine-similarity)
    - [Your turn!](#Your-turn-Similarity)

## Text Preparation

In this section, we explain how to prepare a text for analysis. This includes tokenizing the text, removing stopwords, etc. 

We will use the [spaCy](https://spacy.io/) library, an open-source natural language processing library for Python. It is designed particularly for production use, and it can help us to build applications that process massive volumes of text efficiently.

You can directly [install the library](https://spacy.io/usage) in your Anaconda environment, or, if you opened this notebook in Colab, with the following line of code: 
```python
!pip install -U spacy
```

We also install the English-language model: in you Anaconda environment install "spacy-model-en_core_web_sm"; in Colab, run the following line of code: 
```python
!python -m spacy download en_core_web_sm
```

Note: If you obtain the error `Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed"`, try the following: 
```python
!pip --trusted-host github.com --trusted-host objects.githubusercontent.com install https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0.tar.gz
```

Once everything is installed, and imported (at the beginning of this notebook), we can load our language dictionary, namely the English language model, using `spacy.load('en_core_web_sm')`:

In [ ]:
# Load English language model
sp = spacy.load('en_core_web_sm')

### Tokenization

**Tokenization** is the process of breaking a text into pieces called tokens. A **token** simply refers to an individual part of a sentence having some semantic value. In other words, tokens are the elementary building blocks (words, numbers, characters) in a document. 

SpaCy's tokenizer takes input in form of unicode text and outputs a sequence of token objects. In addition, SpaCy automatically breaks your document into tokens when a document is created using the language model.

There are a couple of different ways we can approach this. The first is called **word tokenization**, which means breaking up the text into individual words. This is a critical step for many language processing applications, as they often require inputs in the form of individual words rather than longer strings of text.

Let’s take a look at a simple example. Imagine we have the following text, and we would like to tokenize it:

> When learning data science, you shouldn't get discouraged!

> Challenges and setbacks aren't failures, they're just part of the journey. You've got this!

We create a spaCy object, which contains linguistic annotations and various language properties:

In [ ]:
# Declare text
text = """When learning data science, you shouldn't get discouraged!
Challenges and setbacks aren't failures, they're just part of the journey. You've got this!"""

# spaCy object is used to create a document
my_doc = sp(text)

my_doc

In [ ]:
# This is a spaCy document
type(my_doc)

Let's now create a list of tokens:

In [ ]:
# Create list of tokens
token_list = [token.text for token in my_doc]
token_list

As we can see, spaCy produces a list that contains each token as a separate item. Notice that it has recognized that contractions such as _shouldn’t_ actually represent two distinct words, and has thus broken them down into two distinct tokens.

We can also see the parts-of-speech (POS) of each of these tokens using the `.pos_` attribute, as shown below.

In [ ]:
# POS
for word in my_doc:
    print(word.text, '->', word.pos_)

POS tagging can be really useful, particularly if you have words or tokens that can have multiple POS tags. For instance, the word "fish" can be used as both a noun and verb, depending upon the context:

In [ ]:
# Another example
doc1 = sp("I like to fish") # verb
doc2 = sp("I eat a fish") # noun

for word in doc1:
    print(word.text, '->', word.pos_)

print("-----------------")

for word in doc2:
    print(word.text, '->', word.pos_)

If we want, we can also break the text into sentences rather than words. This is called **sentence tokenization**. When performing sentence tokenization, the tokenizer looks for specific characters that normally fall between sentences, like periods, exclamation points, and newline characters.

In [ ]:
# create list of sentence tokens
sents_list = [sent.text for sent in my_doc.sents]

sents_list

### Remove Stopwords

Most text data that we work with is going to contain a lot of words that are not actually useful to our analysis (e.g., "is", "and", "you", etc.). These words, called **stopwords**, are useful in human speech, but they do not have much to contribute to the meaning of a sentence. Removing stopwords helps us eliminate noise and distraction from our text data, and also speeds up the time of the analysis (since there are fewer words to process). This makes text analysis more efficient.


Let’s take a look at the stopwords spaCy includes by default.

In [ ]:
# Import stopwords from English language
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

# Print total number of stopwords
print('Number of stopwords: %d' % len(spacy_stopwords))

# Print 20 stopwords
print('20 stopwords: %s' % list(spacy_stopwords)[:20])

Now that we’ve got our list of stopwords, let’s use it to remove the stopwords from the text string we were working on in the previous section.

In [ ]:
# Which words will be removed?
my_doc

In [ ]:
# Filter stopwords
filtered_sent = [word.text for word in my_doc if word.is_stop == False]

print('The filtered sentence contains the words:', filtered_sent)

We can also remove the punctuation:

In [ ]:
# Filter stopwords, punctuation and spaces
filtered_sent2 = []
removed_tokens = []

for word in my_doc:
    if (word.is_stop == True) or (word.is_punct == True) or (word.is_space == True):
        removed_tokens.append(word.text)
    else:
        filtered_sent2.append(word.text)

print('We remove the following tokens:', removed_tokens)
print('The filtered sentence contains the words:', filtered_sent2)

### Lemmatization

**Lemmatization** is a way of dealing with the fact that while words like connect, connection, connecting, connected, etc. are not exactly the same, they all have the same essential meaning: connect. The differences in spelling have grammatical functions in spoken language, but for machine processing, those differences can be confusing, so we need a way to change all the words that are forms of the word connect into the word connect itself.

One method for doing this is called **stemming**. Stemming involves simply lopping off easily-identified prefixes and suffixes to produce what is often the simplest version of a word, the root. Connection, for example, would have the -ion suffix removed and be reduced to connect. This kind of simple stemming is often all that is needed, but lemmatization — which actually looks at words and their roots (called lemma) as described in the dictionary — is more precise (e.g feet -> foot).

Let's look at this simple example.

In [ ]:
# Lemmatization
lem = sp("run runs ran running runner runners")

# Find lemma for each word
for word in lem:
    print(word.text, '->', word.lemma_)

### Your turn! <a id = "Your-turn-Preparation"></a>

The text below is taken from the [E4S report](https://e4s.center/resources/reports/true-cost-of-food-as-a-lever-to-transform-the-swiss-food-system/) on the *True cost of food as a lever to transform the Swiss food system*.

In [ ]:
text = """It is estimated that CHF 37.4 billion is spent every year on food in Switzerland. 
The estimated external costs are at least twice this amount, with health externalities weighing the most. 
Even if these costs are hidden from the market, they are incurred by our planet and societies, through e.g. public health costs and natural resources depletion.
"""

- Create two lists:
    - the first one containing the punctuation and the stopwords,
    - the second one containing the words (tokens).

In [ ]:
# YOUR CODE HERE


- For each token, print its lemma

*Note:* You can convert a list of strings into a string using for instance the `join()` method

In [ ]:
# YOUR CODE HERE


## Text structure

### Dependency Parsing

**Dependency parsing** is a language processing technique that allows to better determine the meaning of a sentence by analyzing how it is constructed to determine how the individual words relate to each other.

Consider, for example, the sentence “Florence throws the ball.” We have two nouns (Florence and ball) and one verb (throws). But we cannot just look at these words individually, or we may end up thinking that the ball throws Florence! To understand the sentence correctly, we need to look at the word order and sentence structure, not just the words.

Below, we have a short sentence. We’ll use a spaCy method called `noun_chunks` ([Documentation](https://spacy.io/usage/linguistic-features#dependency-parse)), which breaks the input down into nouns and the words describing them, and iterate through each chunk in our source text, identifying the word, its root, its dependency identification, and which chunk it belongs to.

In [ ]:
doc = sp("Florence threw a ball, and her friend Edoardo, in pursuit of the ball, hit a wall.")

for chunk in doc.noun_chunks:
    print(chunk.text, '-' , chunk.root.text, '-', chunk.root.dep_, '-', chunk.root.head.text)

Let's visualize this to better understand how our sentence is structured. We use a spaCy visualizer ([Documentation](https://spacy.io/usage/visualizers)):

In [ ]:
# Visualization
displacy.render(doc, style="dep", jupyter= True, options={'distance': 100})

Since most of the tags are pretty abstract (and they vary between languages), you can use `spacy.explain` to get a short description of that the tag stands for:

In [ ]:
tags = ['nsubj', 'dobj', 'det', 'cc', 'conj', 'poss', 'appos', 'prep', 'pobj',
       'PROPN', 'VERB', 'DET', 'NOUN', 'CCONJ', 'ADP']

@interact
def spacy_explain(dep = tags):
    return spacy.explain(dep)

### Entity Detection

Entity detection, also called entity recognition, is a more advanced form of language processing that identifies important elements like places, people, organizations, and languages within a text. This is really helpful for quickly extracting information from the text, since you can quickly pick out important topics or identify key sections of it.

SpaCy, comes with a pre-trained classifier that detects important entities: location, time, people, money etc. To get the named entities from a document, you have to use the `ents` attribute. 

Let’s try out some entity detection using sentences about the life of [Marie Curie](https://en.wikipedia.org/wiki/Marie_Curie):

In [ ]:
article = sp("""Marie Curie was a Polish and naturalized-French physicist and chemist. She was born in Warsaw in 1867 and moved to France in 1891.
She was the first person to be awarded two Nobel Prizes by the Royal Swedish Academy of Sciences: in physics in 1903 for her pioneering research on radioactivity and in chemistry in 1911 for the discovery of the elements polonium and radium.
During World War I she developed mobile radiography units to provide X-ray services to field hospitals.""")

entities = [(i, i.label_, i.label) for i in article.ents]
entities

The above example how spaCy is able to identify a variety of different entity types, including specific locations (GPE), date-related words (DATE), important numbers (CARDINAL), specific individuals (PERSON), etc.

Using `displaCy` we can also visualize the text, with each identified entity highlighted by a color and labeled. We’ll use `style="ent"` to tell displaCy that we want to visualize entities here.

In [ ]:
displacy.render(article, style="ent", jupyter=True)

### Your turn! <a id = "Your-turn-structure"></a>

- Display the entities in the sentences below, about the life of [Elinor Ostrom](https://en.wikipedia.org/wiki/Elinor_Ostrom):

In [ ]:
sentence = """Elinor Olstrom was an American political scientist and political economist born in Los Angeles. In 2009, she became the first woman to receive the Nobel Memorial Prize in Economic Sciences for her analysis of economic governance, especially the commons. 
By conducting field studies, she showed that, contrary to popular beliefs on the tragedy of the commons, natural resources that are collectively used by their users are not over-exploited and destroyed in the long-term. Instead, local communities manage shared natural resources (e.g., pastures, fishing waters, and forests) by establishing rules so that resources are used in a way that is both economically and ecologically sustainable.
Her research is described in her book "Governing the Commons: The Evolution of Institutions for Collective Action".
"""

# YOUR CODE HERE


## Text Representation

The goal is to transform text into numerical features such that it can be used by ML algorithms. There are different techniques:
- **Bag of Words (BOW)** simply treat every document as an unordered set of words. It works in many case but order is not preserved. As a solution, we can use **n-grams**, i.e., we count token pairs, triplets, etc.
- **TF-IDF**: emphasizes important words, i.e., words that appear frequently in a document, (informing about the topic of the document), and words that are rare in a corpus of documents (setting one document apart from other similar ones).

To transform our text, we are going to use our old friend, the scikit learn library. As input it will require strings (and not a spaCy object). Here, our corpus of documents will consist of four sentences on [symbiosis](https://en.wikipedia.org/wiki/Symbiosis). 

In [ ]:
# Sentences (as strings, not spaCy objects)
s1 = "Symbiosis is any type of a close and long-term biological interaction between two biological organisms of different species."
s2 = "Mutualism describes the ecological interaction between two or more species where each species has a net benefit."
s3 = "Commensalism is a long-term biological interaction (symbiosis) in which members of one species gain benefits while those of the other species neither benefit nor are harmed."
s4 = "Parasitism is a close relationship between species, where one organism, the parasite, lives on or inside another organism, the host, causing it some harm, and is adapted structurally to this way of life."

# List of sentences
texts = [s1, s2, s3, s4]
texts

### Bag of Words (BOW)

We use the `CountVectorizer` class of sklearn ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)), using as parameters:
- `ngram_range=(1,2)`, i.e., we consider tokens (1-grams) and pair of tokens (2-grams);
- `stop_words="english"`, a built-in stop word list for English.

In [ ]:
# Using default tokenizer 
count = CountVectorizer(ngram_range=(1,2), stop_words="english")

# Learn the vocabulary dictionary and return document-term matrix
bow = count.fit_transform(texts)

# Show feature matrix
print(bow.toarray())

Let's check the n-grams (tokens and pair of tokens) created:

In [ ]:
# Get feature names
feature_names = count.get_feature_names_out()

# View feature names
print('Our n-grams are:', ', '.join(feature_names))

We can better visualize the result in a dataframe:

In [ ]:
# Show as a dataframe
pd.set_option("display.max_columns", None)
pd.DataFrame(
    bow.todense(),              # Feature matrix
    columns=feature_names,      # n-grams
    index= ['s1', 's2', 's3', 's4']
    )

### TF-IDF Representation

**TF-IDF** emphasizes important words. It is the product of term frequency (TF) and inverse document frequency (IDF):
- **Term Frequency** identifies tokens that appear frequently in a document: TF(token, document) = number of times token appears in document / total number of tokens in document
- **Inverse Document Frequency** identifies words that appear rarely in the corpus: IDF(token, corpus) = log( total number of documents in corpus / number of documents containing token )

Note that the IDF value for a token remains the same throughout all the documents as it depends upon the total number of documents. On the other hand, TF values of a token differ from document to document.

The goal of using TF-IDF instead of the raw frequencies of occurrence of a token in a given document is to scale down the impact of tokens that occur very frequently in a given corpus since those are less informative than tokens that occur in a small fraction of the corpus. 

Ok, let's implement TD-IDF. We are using the `TfidfVectorizer` class of sklearn ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer)):

In [ ]:
# Using default tokenizer in TfidfVectorizer
tfidf = TfidfVectorizer(ngram_range=(1, 1), stop_words="english")

# Learn the vocabulary dictionary and return document-term matrix
features = tfidf.fit_transform(texts)

# Visualize result in dataframe
pd.DataFrame(
    features.todense(),
    columns=tfidf.get_feature_names_out(),
    index = ['s1', 's2', 's3', 's4']
)

### Your turn! <a id = "Your-turn-Representation"></a>

- Create a TF-IDF Representation of the three sentences below using bigrams

In [ ]:
# Sentences
s1 = "The linear economy, sometimes referred to as the take-make-waste economy, is a system where resources are extracted to make products that eventually end up as waste and are thrown away."
s2 = "A circular economy is a regenerative economic system that uses renewable energy and resources, reuses materials and products as long as possible, and recycles resources rather than disposing them as waste."
s3 = "A green economy is an economy that aims at reducing environmental risks and ecological scarcities, and that aims for sustainable development without degrading the environment."

In [ ]:
# YOUR CODE HERE


## Application: Text Similarity <a id = "Your-turn-Similarity"></a>

In this application, we will compute the similarity between various texts. We'll use Wikipedia articles, relying on the `wikipedia` library ([Documentation](https://pypi.org/project/wikipedia/)) to avoid doing the preprocessing, e.g., web scrapping to extract the articles. You can install the library using `pip install wikipedia`.

In [ ]:
topics = ["Albert Einstein",
          "Adam Smith",
          "Alexander von Humboldt",
          "Amartya Sen",
          "Daniel Kahneman",
          "John von Neumann",
          "Joseph Stiglitz",
          "Stephen Hawking"
]

# Extract summary of articles
data_wiki = [wikipedia.summary(topics[p], sentences=20) for p in range(len(topics))]

Let's learn a bit more about [Alexander von Humboldt](https://en.wikipedia.org/wiki/Alexander_von_Humboldt):

In [ ]:
data_wiki[2]

The "father of ecology", not bad! Ok, let's start our similarity analysis.

### Converting text into vectors

The first step is to convert our articles into numerical features. We use `CountVectorizer`, i.e., the Bag of Words technique, with tokens and pair of tokens (2-grams), and without stopwords.

In [ ]:
# Using default tokenizer 
count_wiki = CountVectorizer(ngram_range=(1,2), stop_words = None)
bow_wiki = count_wiki.fit_transform(data_wiki)

### Cosine similarity

We can now compute the **cosine similarity**. We use the `cosine_similarity` module of sklearn ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html#sklearn.metrics.pairwise.cosine_similarity)).

In [ ]:
# Compute pairwise similarities
pairwise_similarities = cosine_similarity(bow_wiki, bow_wiki)

# Visualize result in dataframe
pairwise_df = pd.DataFrame(
    pairwise_similarities,
    columns = topics,
    index = topics
)
pairwise_df

Let's plot a heatmap to obtain a more visual representation:

In [ ]:
# Heatmap of pairwise similarities
plt.figure(figsize=(15,5))
sns.heatmap(
    pairwise_df,
    cmap='OrRd',
    linewidth=1,
    annot=True
)
plt.show()

### Your turn!

Let's continue the exploration of similarities between our articles.

- Compute a heatmap of the pairwise similarities, varying the vectorization technique (`CountVectorizer`, `TfidfVectorizer`), the ngram range, and the stopwords (e.g., `None`, "english"). How do your model compares?

*Note:* you could define a function and try interactive features.

In [ ]:
# YOUR CODE HERE


- Repeat the previous operations, but this time using the full text for each Wikipedia entry. What do you observe?

In [ ]:
data_full_wiki = [wikipedia.page(topics[p]).content for p in range(len(topics))]

In [ ]:
# YOUR CODE HERE
